<a href="https://colab.research.google.com/github/venu72561-chinnam/AI-ML-project/blob/main/LLMarchitecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install transformers datasets torch accelerate gradio -q

In [22]:
from transformers import (
    AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling,
    Trainer, TrainingArguments, T5ForConditionalGeneration, T5Tokenizer, pipeline
)
from datasets import Dataset
import gradio as gr
import torch

In [23]:
sample_text = "Large Language Models generate human-like text."

# Tokenizer and model (DistilBERT for simplicity)
tokenizer_demo = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model_demo = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased")

In [24]:
tokens = tokenizer_demo.tokenize(sample_text)
print(" Tokens:", tokens)

 Tokens: ['large', 'language', 'models', 'generate', 'human', '-', 'like', 'text', '.']


In [25]:
input_ids = tokenizer_demo(sample_text, return_tensors="pt").input_ids
embeddings = model_demo.distilbert.embeddings.word_embeddings(input_ids)
print("Embedding shape:", embeddings.shape)

Embedding shape: torch.Size([1, 11, 768])


In [26]:
data = [
    {"instruction": "Summarize the given text.",
     "input": "Machine learning is a field of AI that enables systems to learn from data.",
     "output": "Machine learning enables systems to learn from data."},
    {"instruction": "Explain tokenization.",
     "input": "",
     "output": "Tokenization splits text into smaller units like words or subwords."},
    {"instruction": "Define embedding.",
     "input": "",
     "output": "Embedding represents words as vectors capturing their meaning."}
]

In [27]:
dataset = Dataset.from_list(data)

In [28]:
def preprocess_mlm(example):
    text = f"{example['instruction']} {example['input']} {example['output']}"
    return tokenizer_demo(text, truncation=True, padding="max_length", max_length=64)

In [29]:
tokenized_dataset = dataset.map(preprocess_mlm, batched=False)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer_demo, mlm=True, mlm_probability=0.15
)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [30]:
!pip install transformers datasets sentencepiece accelerate torch gradio -q


import os
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"
os.environ["TRANSFORMERS_OFFLINE"] = "0"   # allow model download, but no login
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["WANDB_DISABLED"] = "true"      # disables wandb tracking
os.environ["HF_HOME"] = "/tmp"             # use temp storage
os.environ["HF_DATASETS_OFFLINE"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

#  Imports
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments, pipeline
from datasets import Dataset
import gradio as gr

# Create a small instruction dataset
data = [
    {"instruction": "Summarize the given text.",
     "input": "Machine learning is a field of artificial intelligence that enables systems to learn from data and improve automatically.",
     "output": "Machine learning enables systems to learn and improve from data."},
    {"instruction": "Explain what tokenization means in NLP.",
     "input": "",
     "output": "Tokenization is the process of splitting text into smaller units like words or subwords."},
    {"instruction": "Define embedding in simple terms.",
     "input": "",
     "output": "An embedding is a way to represent words as numerical vectors capturing their meaning."},
    {"instruction": "Summarize the text.",
     "input": "Artificial Intelligence enables machines to perform tasks that normally require human intelligence.",
     "output": "AI enables machines to mimic human intelligence."},
]
dataset = Dataset.from_list(data)

# Tokenization / preprocessing
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name, use_auth_token=False)

def preprocess(example):
    inp = f"Instruction: {example['instruction']} Input: {example['input']}"
    model_inputs = tokenizer(inp, truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(example["output"], truncation=True, padding="max_length", max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_data = dataset.map(preprocess, batched=False)

#  Model & training (fine-tuning on small data)
model = T5ForConditionalGeneration.from_pretrained(model_name, use_auth_token=False)

training_args = TrainingArguments(
    output_dir="./t5_instruction_finetune",
    learning_rate=3e-5,
    per_device_train_batch_size=2,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    save_strategy="no",
    push_to_hub=False,        # <— prevents token prompt
    report_to=[]              # <— disables any reporting
)

trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_data)
trainer.train()
trainer.save_model("./t5_instruction_model")

#  Inference pipeline
pipe = pipeline("text2text-generation", model="./t5_instruction_model", tokenizer=model_name)

#  Gradio Interface
def generate_text(instruction, user_input):
    prompt = f"Instruction: {instruction} Input: {user_input}"
    result = pipe(prompt, max_length=80)[0]['generated_text']
    return result

demo = gr.Interface(
    fn=generate_text,
    inputs=[gr.Textbox(label="Instruction (e.g. Summarize the text)"),
            gr.Textbox(label="Input Text")],
    outputs=gr.Textbox(label="Generated Output"),
    title=" Mini GenAI Instruction Model (Token-Free)",
    description="Fine-tuned T5-small model trained"
)

demo.launch(share=True)

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1908: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:4841: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Step,Training Loss


Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://796c2900e5a6f2eb9a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
